风格迁移是指将一幅图像的风格迁移到另一幅图像上，使两幅图像具有相同的风格。不会改变原图的主体内容，但会使两幅图像具有相似的外观。风格迁移技术可以应用于多种领域，如照片美化、视频特效、游戏美化、广告创意等。
![image.png](attachment:image.png)

## CNN的层次化特征表示
通过多层卷积逐渐提取不同层次的特征，浅层卷积捕捉局部基础特征，如颜色，边缘， 纹理等，而深层卷积捕捉全局特征，如物体的形状，整体结构等。

## 风格迁移的原理
风格迁移的本质是通道的相关性，通过Gram矩阵来量化，其丢弃特征的空间位置信息，只保留通道之间的统计关系（相关性）
### 计算步骤
1. 对某一卷积层的输出特征图将特征图从三维张量转换为二维矩阵，保留通道维度 :

$F \in \mathbb{R}^{C \times H \times W}$展开为 $F \in \mathbb{R}^{C \times (H \cdot W)}$

2. 计算Gram矩阵 

Gram矩阵是矩阵定义为特征矩阵与其转置的乘积，而特征矩阵就是原图展开成二维张量后的矩阵

`
features = input.view(batch * channel, h * w)
gram = torch.mm(features, features.t()) `

矩阵乘法：$G_{ij} = \sum_{k=1}^{HW} f_{i,k} \cdot f_{j,k}$， $G_{ij}$ 表示通道 $i$ 和 $j$ 的特征相关性，而对角线元素 $G_{ii}$ 表示通道 $i$ 的激活强度的平方和，即此通道的能量。

对角线元素：表示某个特征的强度（比如“红色通道的能量”）。
非对角线元素：表示不同特征的协同关系（比如“红色和绿色是否经常一起出现”）。

$G = F \cdot F^T \in \mathbb{R}^{C \times C}$，

3. 归一化,Gram矩阵除以特征图的总元素数：$G \leftarrow G / (C \cdot H \cdot W)$，消除特征图尺寸影响。如果不除，原图的颜色会变成风格图像的颜色，原因在于若是不除，其相应的值会变大，也就是风格图像的统计信息会被放大，这些过大的值会主导优化过程，导致生成图像倾向于复制风格图像的颜色分布。

Gram矩阵让内容和风格分离，对角线元素 $G_{ii}$：通道 $i$ 的激活强度的平方和（即该通道的“能量”）。

非对角线元素 $G_{ij}$：通道 $i$ 和 $j$ 的共现模式。

正值：两通道倾向于同时激活（如红色和黄色在火焰风格中常共存）。

负值：两通道抑制彼此（如某些颜色互斥）。

风格迁移中的作用：
Gram矩阵丢弃了特征的空间位置信息（即纹理的绝对位置），但保留了通道间的统计关系，这正是“风格”的本质

### Gram矩阵的问题
1. Gram矩阵的原始意义与问题
Gram矩阵的原始计算（未归一化）为：
$G_{ij} = \sum_{k=1}^{HW} f_{i,k} \cdot f_{j,k}$
 
其中 $F \in \mathbb{R}^{C \times (H \cdot W)}$ 是展开后的特征图。

问题：$G_{ij}$ 的值直接依赖于特征图的尺寸（$H \cdot W$）和通道激活的绝对值大小。

若 $H \cdot W$ 很大（如深层特征图尺寸较小，浅层较大），浅层的Gram矩阵值会天然远大于深层。

若某一通道的激活值普遍较大（如风格图像中红色通道主导），该通道在Gram矩阵中的权重会过高。

2. 不归一化的后果
(1) 颜色通道的主导性
假设风格图像的红色通道（$F_{\text{red}}$）激活值远大于其他通道（如蓝色通道 $F_{\text{blue}}$）：

未归一化的Gram矩阵中，$G_{\text{red,red}}$ 的值会极大，而 $G_{\text{blue,blue}}$ 的值极小。

在优化风格损失时，模型会优先匹配红色通道的相关性（因为其损失项数值更大），导致生成图像的红色部分完全覆盖内容图像的颜色。

(2) 特征图层级的失衡
浅层特征图（如conv1）的尺寸 $H \cdot W$ 通常远大于深层（如conv4）：

未归一化时，浅层的Gram矩阵值更大，风格损失会被浅层特征主导。

深层特征对内容的约束会被淹没，生成图像可能丢失内容结构，仅保留风格图像的纹理和颜色。

